In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
import re
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from scipy.stats import sem
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.15f}'.format
import datetime as dt

In [2]:
data = pd.read_csv('final_1416.csv')
data.head()

,type,rcvTime,sendTime,sender,senderPseudo,messageID,file_name,density_level,pos_x,pos_y,...,acl_noise_y,acl_noise_z,hed_x,hed_y,hed_z,hed_noise_x,hed_noise_y,hed_noise_z,attack?,receiver_id
0,3,57511.049951916953432,57511.049951916953432,9891.000000000000000,1098915.000000000000000,19655176.000000000000000,traceJSON-10005-10003-A0-57511-15,2,137.286902555145161,983.827257071713575,...,0.000021258383547,0.000000000000000,-0.036686450374153,0.999326825597584,0.000000000000000,4.249188192325861,0.982232769632962,0.000000000000000,1,10005
1,3,57511.146510437574761,57511.146510437574761,9951.000000000000000,1099515.000000000000000,19655429.000000000000000,traceJSON-10005-10003-A0-57511-15,2,153.904668561966304,899.505911646706977,...,0.000000000000013,0.000000000000000,0.038287039242177,-0.999266782509089,0.000000000000000,9.636084890334937,12.342153049505139,0.000000000000000,1,10005
2,3,57511.162338829126384,57511.162338829126384,10011.000000000000000,10100115.000000000000000,19655584.000000000000000,traceJSON-10005-10003-A0-57511-15,2,142.981000552607952,1038.129049878944215,...,0.000424009628911,0.000000000000000,0.073948678480006,-0.997262048285735,0.000000000000000,7.179251774207310,7.627622845974860,0.000000000000000,1,10005
3,3,57511.590730855474249,57511.590730855474249,9957.000000000000000,1099575.000000000000000,19656248.000000000000000,traceJSON-10005-10003-A0-57511-15,2,149.931080349069163,899.035172728899624,...,0.000000000062721,0.000000000000000,0.016440994151280,-0.999864837721238,0.000000000000000,16.013310920410738,21.388849080932467,0.000000000000000,0,10005
4,3,57511.641619857240585,57511.641619857240585,9963.000000000000000,1099635.000000000000000,19656466.000000000000000,traceJSON-10005-10003-A0-57511-15,2,152.753405416050725,906.773024725052210,...,0.000000000000000,0.000000000000000,0.047863865685627,-0.998853868371959,0.000000000000000,6.510268774099048,7.224761247950584,0.000000000000000,0,10005


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820097 entries, 0 to 820096
Data columns (total 34 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   type           820097 non-null  int64  
 1   rcvTime        820097 non-null  float64
 2   sendTime       820097 non-null  float64
 3   sender         820097 non-null  float64
 4   senderPseudo   820097 non-null  float64
 5   messageID      820097 non-null  float64
 6   file_name      820097 non-null  object 
 7   density_level  820097 non-null  int64  
 8   pos_x          820097 non-null  float64
 9   pos_y          820097 non-null  float64
 10  pos_z          820097 non-null  float64
 11  pos_noise_x    820097 non-null  float64
 12  pos_noise_y    820097 non-null  float64
 13  pos_noise_z    820097 non-null  float64
 14  spd_x          820097 non-null  float64
 15  spd_y          820097 non-null  float64
 16  spd_z          820097 non-null  float64
 17  spd_noise_x    820097 non-nul

In [4]:
from datetime import datetime
#Dropping the columns that are of no use
data.drop(['type'],axis=1,inplace=True)
data.drop(['pos_z'],axis=1,inplace=True)
data.drop(['spd_z'],axis=1,inplace=True)
data.drop(['acl_z'],axis=1,inplace=True)
data.drop(['hed_z'],axis=1,inplace=True)
data['rcvTime_numeric'] = data['rcvTime']
data['sendTime_numeric'] = data['sendTime']
data[['rcvTime','sendTime']]  = data[['rcvTime','sendTime']].apply(pd.to_datetime)
data[['sender','messageID']]  = data[['sender','messageID']].fillna(0.0).astype(int)
data.rename(columns = {'attack?':'attack'}, inplace = True)

#Dropping the columns with noise
for i in data.columns:
    if 'noise' in i:
        data.drop(i, axis=1, inplace=True)

In [5]:
data.drop(['file_name'], axis=1, inplace=True)
data['sendTime_numeric']=data['sendTime'].map(dt.datetime.toordinal)
data['rcvTime_numeric']=data['rcvTime'].map(dt.datetime.toordinal)

## Sorting data by receive time and splitting data for training

In [6]:
# Sorting by receive time
data.sort_values(by=['rcvTime'], inplace=True)
data.reset_index(drop=True, inplace=True)
data.head()

,rcvTime,sendTime,sender,senderPseudo,messageID,density_level,pos_x,pos_y,spd_x,spd_y,acl_x,acl_y,hed_x,hed_y,attack,receiver_id,rcvTime_numeric,sendTime_numeric
0,1970-01-01 00:00:00.000050408,1970-01-01 00:00:00.000050408,21,10215.000000000000000,16936,2,220.317917303648272,383.934935437669594,-0.078168163622794,0.525905153291198,-0.388921475468878,2.616637002617828,-0.142980189511322,0.989725550547881,0,27,719163,719163
1,1970-01-01 00:00:00.000050408,1970-01-01 00:00:00.000050408,27,10275.000000000000000,17018,2,223.030520488101104,364.802884416285281,-0.079870635180307,0.537359158655796,-0.388233666394645,2.613037040001825,-0.147020325879710,0.989133471164647,0,21,719163,719163
2,1970-01-01 00:00:00.000050409,1970-01-01 00:00:00.000050409,21,10215.000000000000000,17075,2,219.947049702855480,385.757716338412308,-0.421900962680975,2.838498011407711,-0.322469854764553,2.169566146413223,-0.142233817434146,0.989833087534514,0,27,719163,719163
3,1970-01-01 00:00:00.000050409,1970-01-01 00:00:00.000050409,27,10275.000000000000000,17117,2,222.765622516565202,366.657630560278733,-0.419577828663898,2.822925004092247,-0.360335561439105,2.425224516331284,-0.142064310031641,0.989857430044970,0,21,719163,719163
4,1970-01-01 00:00:00.000050410,1970-01-01 00:00:00.000050410,9,1095.000000000000000,17151,2,1197.362034797699152,971.768939400884392,-11.762703940887320,-2.110692806707845,-1.783526867056106,-0.319738065392786,-0.992752282476080,-0.120178640525395,0,15,719163,719163


In [7]:
# Splitting the data into 80% as train and rest(20%) as test
total_size = len(data)
total_size
train_size = round(total_size*.8)
test_size = total_size - train_size
print('total', total_size, 'train_size', train_size, 'test_size', test_size)

total 820097 train_size 656078 test_size 164019


In [8]:
# Populating train and test data based on the index calculated in above cell
train_data = data[0:train_size]
test_data = data[train_size:train_size+test_size]
print('train_data_size', len(train_data), 'test_data_size', len(test_data))

train_data_size 656078 test_data_size 164019


In [9]:
# 12 features
X_train_12 = train_data[['messageID','sender','senderPseudo','pos_x','pos_y','spd_x','spd_y','acl_x','acl_y', 'hed_x','hed_y','rcvTime_numeric']]
X_test_12 = test_data[['messageID','sender','senderPseudo','pos_x','pos_y','spd_x','spd_y','acl_x','acl_y', 'hed_x','hed_y','rcvTime_numeric']]
y_train_12 = train_data[['attack']]
y_test_12 = test_data[['attack']]

In [10]:
# 11 features
X_train_11 = train_data[['messageID','senderPseudo','pos_x','pos_y','spd_x','spd_y','acl_x','acl_y', 'hed_x','hed_y','rcvTime_numeric']]
X_test_11 = test_data[['messageID','senderPseudo','pos_x','pos_y','spd_x','spd_y','acl_x','acl_y', 'hed_x','hed_y','rcvTime_numeric']]
y_train_11 = train_data[['attack']]
y_test_11 = test_data[['attack']]

In [11]:
# 10 features
X_train_10 = train_data[['messageID','pos_x','pos_y','spd_x','spd_y','acl_x','acl_y', 'hed_x','hed_y','rcvTime_numeric']]
X_test_10 = test_data[['messageID','pos_x','pos_y','spd_x','spd_y','acl_x','acl_y', 'hed_x','hed_y','rcvTime_numeric']]
y_train_10 = train_data[['attack']]
y_test_10 = test_data[['attack']]

In [12]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train_12,y_train_12)

#Predict the response for test dataset
y_pred = clf.predict(X_test_12)
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",accuracy_score(y_test_12, y_pred))

Accuracy: 0.9833555868527427


**RandomForestClassifier**

Simple RandomForest classifer without hyperparameters - 12 features


In [13]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train_12, y_train_12)
y_pred_12 = clf.predict(X_test_12)
print(classification_report(y_test_12,y_pred_12))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93    114796
           1       1.00      0.67      0.80     49223

    accuracy                           0.90    164019
   macro avg       0.94      0.84      0.87    164019
weighted avg       0.91      0.90      0.89    164019



In [14]:
#Simple RandomForest classifer without hyperparameters - 11 features
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train_11, y_train_11)
y_pred_11 = clf.predict(X_test_11)
print(classification_report(y_test_11,y_pred_11))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95    114796
           1       1.00      0.76      0.86     49223

    accuracy                           0.93    164019
   macro avg       0.95      0.88      0.91    164019
weighted avg       0.93      0.93      0.92    164019



In [15]:
#Simple RandomForest classifer without hyperparameters - 10 features
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train_10, y_train_10)
y_pred_10 = clf.predict(X_test_10)
print(classification_report(y_test_10,y_pred_10))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95    114796
           1       1.00      0.76      0.87     49223

    accuracy                           0.93    164019
   macro avg       0.95      0.88      0.91    164019
weighted avg       0.94      0.93      0.93    164019



In [16]:
#Simple RandomForest classifer with hyperparameters - 12 features
clf = RandomForestClassifier(random_state=0, criterion='entropy',max_depth=7, max_features='auto',n_estimators= 50)
clf.fit(X_train_12, y_train_12)
y_pred_12 = clf.predict(X_test_12)
print(classification_report(y_test_12,y_pred_12))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    114796
           1       1.00      0.94      0.97     49223

    accuracy                           0.98    164019
   macro avg       0.99      0.97      0.98    164019
weighted avg       0.98      0.98      0.98    164019



In [17]:
#Simple RandomForest classifer with hyperparameters - 11 features
clf = RandomForestClassifier(random_state=0, criterion='entropy',max_depth=7, max_features='auto',n_estimators= 50)
clf.fit(X_train_11, y_train_11)
y_pred_11 = clf.predict(X_test_11)
print(classification_report(y_test_11,y_pred_11))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    114796
           1       1.00      0.94      0.97     49223

    accuracy                           0.98    164019
   macro avg       0.99      0.97      0.98    164019
weighted avg       0.98      0.98      0.98    164019



In [18]:
#Simple RandomForest classifer with hyperparameters - 10 features
clf = RandomForestClassifier(random_state=0, criterion='entropy',max_depth=7, max_features='auto',n_estimators= 50)
clf.fit(X_train_11, y_train_11)
y_pred_11 = clf.predict(X_test_11)
print(classification_report(y_test_11,y_pred_11))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    114796
           1       1.00      0.94      0.97     49223

    accuracy                           0.98    164019
   macro avg       0.99      0.97      0.98    164019
weighted avg       0.98      0.98      0.98    164019



In [20]:
#KNN
#12 features
#Create KNN Classifier
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=6)

#Train the model using the training sets
knn.fit(X_train_12, y_train_12)

#Predict the response for test dataset
y_pred_12 = knn.predict(X_test_12)
print(classification_report(y_test_12,y_pred_12))

              precision    recall  f1-score   support

           0       0.70      1.00      0.82    114796
           1       0.23      0.00      0.00     49223

    accuracy                           0.70    164019
   macro avg       0.46      0.50      0.41    164019
weighted avg       0.56      0.70      0.58    164019



In [21]:
#11 features
#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=6)

#Train the model using the training sets
knn.fit(X_train_11, y_train_11)

#Predict the response for test dataset
y_pred_11 = knn.predict(X_test_11)
print(classification_report(y_test_11,y_pred_11))

              precision    recall  f1-score   support

           0       0.70      1.00      0.82    114796
           1       0.23      0.00      0.00     49223

    accuracy                           0.70    164019
   macro avg       0.46      0.50      0.41    164019
weighted avg       0.56      0.70      0.58    164019



In [22]:
#10 features
#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=6)

#Train the model using the training sets
knn.fit(X_train_10, y_train_10)

#Predict the response for test dataset
y_pred_10 = knn.predict(X_test_10)
print(classification_report(y_test_10,y_pred_10))

              precision    recall  f1-score   support

           0       0.70      1.00      0.82    114796
           1       0.22      0.00      0.00     49223

    accuracy                           0.70    164019
   macro avg       0.46      0.50      0.41    164019
weighted avg       0.56      0.70      0.58    164019



In [25]:
#Logistic Regression
#Simple Logistic Regression
from sklearn.linear_model import LogisticRegression
#Create LogisticRegression Classifier
lr = LogisticRegression()
#Train the model using the training sets
lr.fit(X_train_11, y_train_11)

#Predict the response for test dataset
y_pred_11 = lr.predict(X_test_11)
print(classification_report(y_test_11,y_pred_11))

              precision    recall  f1-score   support

           0       0.70      1.00      0.82    114796
           1       0.00      0.00      0.00     49223

    accuracy                           0.70    164019
   macro avg       0.35      0.50      0.41    164019
weighted avg       0.49      0.70      0.58    164019



In [28]:
#LogisticRegression - with best hyper-parameters found 

#Create LogisticRegression Classifier
lr = LogisticRegression(C= 10, penalty= 'l2', solver= 'newton-cg')
#Train the model using the training sets
lr.fit(X_train_11, y_train_11)

#Predict the response for test dataset
y_pred_11 = lr.predict(X_test_11)
print(classification_report(y_test_11,y_pred_11))
print(accuracy_score(y_test_11,y_pred_11))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97    114796
           1       0.98      0.89      0.94     49223

    accuracy                           0.96    164019
   macro avg       0.97      0.94      0.96    164019
weighted avg       0.96      0.96      0.96    164019

0.9641504947597535


In [27]:
#Naive Bayes
#12 features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train_12)
X_test = sc.transform(X_test_12)
from sklearn.naive_bayes import GaussianNB
classifier1 = GaussianNB()
classifier1.fit(X_train_12, y_train_12)
GaussianNB()
y_pred_12  =  classifier1.predict(X_test_12)
#from sklearn.metrics import confusion_matrix,accuracy_score
#cm = print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test_12,y_pred_12))
print(classification_report(y_test_12, y_pred_12))

0.6914016059115102
              precision    recall  f1-score   support

           0       0.70      0.98      0.82    114796
           1       0.28      0.02      0.03     49223

    accuracy                           0.69    164019
   macro avg       0.49      0.50      0.43    164019
weighted avg       0.57      0.69      0.58    164019



In [30]:
#Naive Bayes
#11 features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train_11)
X_test = sc.transform(X_test_11)
from sklearn.naive_bayes import GaussianNB
classifier1 = GaussianNB()
classifier1.fit(X_train_11, y_train_11)
GaussianNB()
y_pred_11  =  classifier1.predict(X_test_11)
#from sklearn.metrics import confusion_matrix,accuracy_score
#cm = print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test_11,y_pred_11))
print(classification_report(y_test_11, y_pred_11))

0.6914016059115102
              precision    recall  f1-score   support

           0       0.70      0.98      0.82    114796
           1       0.28      0.02      0.03     49223

    accuracy                           0.69    164019
   macro avg       0.49      0.50      0.43    164019
weighted avg       0.57      0.69      0.58    164019



In [31]:
#Naive Bayes
#10 features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train_10)
X_test = sc.transform(X_test_10)
from sklearn.naive_bayes import GaussianNB
classifier1 = GaussianNB()
classifier1.fit(X_train_10, y_train_10)
GaussianNB()
y_pred_10 =  classifier1.predict(X_test_10)
#from sklearn.metrics import confusion_matrix,accuracy_score
#cm = print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test_10,y_pred_10))
print(classification_report(y_test_10, y_pred_10))

0.6998945244148543
              precision    recall  f1-score   support

           0       0.70      1.00      0.82    114796
           1       0.00      0.00      0.00     49223

    accuracy                           0.70    164019
   macro avg       0.35      0.50      0.41    164019
weighted avg       0.49      0.70      0.58    164019

